# Creating behavior reports
will create a summary of a behavior session.

In [ ]:
import json
import os
from os.path import join
from src import behavior_class as bc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
mouse_id = "140302_3"

In [ ]:
behavior = bc.behaviorData(mouse_id)
behavior_folders = [
    (index, folder) for index, folder in enumerate(behavior.find_behavior_folders())
]
behavior_folders

In [ ]:
beh_folder = behavior_folders[0][1]
processed_json = "140302_3_20240118121644_932_sleep.json"

file_path = join(beh_folder, processed_json)

if os.path.isfile(file_path):
    print("File exists")
else:
    print(f"File {processed_json} does not exist")

## Summary data

In [ ]:
with open(join(beh_folder, processed_json)) as f:
    beh_data = json.load(f)

summary_dict = {
    "recording_duration": beh_data[0]["data"]["recordingDuration"],
    "number_laps": max(beh_data[0]["data"]["lap_bin"]),
    "mean_velocity": np.mean(beh_data[0]["data"]["velocity"])
    / beh_data[0]["data"]["recordingDuration"],
}
with open(join(beh_folder, "behavior_summary.json"), "w") as f:
    json.dump(summary_dict, f, indent=4)

## Summary plots

In [ ]:
filtered_velocity = pd.read_json(join(beh_folder, "filtered_velocity.json"))
mob_immob = pd.read_json(join(beh_folder, "mobility_immobility.json"))

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 6), sharex=True)
duration = int(beh_data[0]["data"]["recordingDuration"])
num_data_points = len(beh_data[0]["data"]["velocity"])
x = np.linspace(0, duration, num_data_points)

# Velocity
ax[0, 0].plot(x, np.array(beh_data[0]["data"]["velocity"]))
ax[0, 0].set_title("Velocity")

# position
ax[1, 0].plot(x, np.array(beh_data[0]["data"]["treadmillPosition"]))
ax[1, 0].set_title("Position")
ax[1, 0].set_xlabel("Time [s]")

# filtered velocity
ax[0, 1].plot(x, np.array(filtered_velocity))
ax[0, 1].set_title("Filtered Velocity")

# mobility immobility
ax[1, 1].plot(x, mob_immob)
ax[1, 1].set_title("Mobility Immobility")
ax[1, 1].set_xlabel("Time [s]")

plt.tight_layout()
plt.savefig(join(beh_folder, "behavior_summary.png"))

In [ ]:
plt.plot(np.array(beh_data[0]["data"]["velocity"]))
plt.plot(np.array(filtered_velocity))